In [23]:
# 필요한 모듈 import
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import os

In [24]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv, find_dotenv

# API KEY 정보로드
load_dotenv(find_dotenv())

True

In [26]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [18]:
########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("고용보험법 시행령(대통령령)(제34048호)(20240101).pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

In [19]:
split_docs

[Document(page_content='법제처                                                            1                                                       국가법령정보센터\n고용보험법 시행령 \n \n고용보험법 시행령 \n[시행 2024. 1. 1.] [대통령령 제34048호, 2023. 12. 26., 일부개정] \n고용노동부 (고용보험기획과 - 고용보험제도) 044-202-7352 \n고용노동부 (여성고용정책과 - 모성보호) 044-202-7476 \n고용노동부 (인적자원개발과 - 사업주 및 근로자 직업능력개발 훈련지원) 044-202-7317 \n고용노동부 (고용보험기획과) 044-202-7373 \n고용노동부 (고용정책총괄과 - 고용촉진지원금, 고용유지지원금 등) 044-202-7218 \n고용노동부 (고용지원실업급여과 - 피보험자 관리) 044-202-7378 \n고용노동부 (고용지원실업급여과 - 실업급여) 044-202-7376 \n       제1장 총칙\n \n제1조(목적) 이 영은 「고용보험법」에서 위임된 사항과 그 시행에 필요한 사항을 규정하는 것을 목적으로 한다.\n \n제1조의2(보수에서 제외되는 금품) 「고용보험법」(이하 “법”이라 한다) 제2조제5호 본문에서 “대통령령으로 정하는 금\n품”이란 「소득세법」 제12조제3호에 따른 비과세 근로소득을 말한다. <개정 2021. 6. 8.>\n[본조신설 2010. 12. 31.]\n[종전 제1조의2는 제1조의3으로 이동 <2010. 12. 31.>]\n \n제1조의3(고용보험위원회의 구성) ① 법 제7조제4항제1호 및 제2호에 따른 근로자와 사용자를 대표하는 사람은 각각\n전국 규모의 노동단체와 전국 규모의 사용자단체에서 추천하는 사람 중에서 고용노동부장관이 위촉한다. <개정\n2010. 7. 12., 2010. 12. 31.>\n② 법 제7조제4항제3호에 따른 공익을 대표하는 사람은 고용보험과

In [27]:
# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

# Retriever를 생성합니다.
retriever = vector.as_retriever()

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="2024년 1월 고용보험법 시행령(대통령령) 제34048호 관련 정보를 PDF 문서에서 검색합니다. '고용보험법 시행령' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-LN1UM***************************************TCLr. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

In [ ]:
########## 2. LLM 을 정의 ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

In [ ]:
########## 3. Prompt 정의 ##########

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

In [ ]:
########## 4. Agent 를 정의합니다 ##########

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# 채팅 메시지 기록을 관리하는 객체를 생성합니다.
message_history = ChatMessageHistory()

In [ ]:
# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대부분의 실제 시나리오에서 세션 ID가 필요하기 때문에 이것이 필요합니다
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    lambda session_id: message_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [ ]:
########## 7. 질의-응답 테스트를 수행합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {"input": "외국인 관련 법 설명해줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-LN1UM***************************************TCLr. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}